In [66]:
import time
import requests
import pandas as pd

athletes = pd.read_csv('data/usernames/accounts_final.csv')

api_key = '4eff157235e983a5f72db09ead7ecf00'
base_url = 'https://gnews.io/api/v4/search'

In [67]:
def fetch_articles(query, api_key, retries=3, delay=5):
    url = f"https://gnews.io/api/v4/search?q={query}&from=2020-01-01&to=2021-12-31&lang=en&max=10&apikey={api_key}"
    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            return data.get('articles', [])
        except (requests.RequestException, ValueError) as e:
            print(f"Error fetching articles for {query}: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
    return []


In [78]:
def process_athletes_in_chunks(athletes_df, api_key, chunk_size=50, max_athletes=1000, output_file='articles.csv'):
    if max_athletes is None:
        max_athletes = len(athletes_df)
    
    all_articles = []
    for start in range(0, min(max_athletes, len(athletes_df)), chunk_size):
        end = min(start + chunk_size, len(athletes_df))
        chunk = athletes_df.iloc[start:end]
        print(f"Processing chunk from index {start} to {end}...")
        
        for athlete in chunk['name']:
            print(f"Fetching articles for {athlete}...")
            articles = fetch_articles(athlete, api_key)
            
            for article in articles:
                all_articles.append({
                    'athlete': athlete,
                    'title': article['title'],
                    'description': article['description'],
                    'url': article['url'],
                    'publish_date': article['publishedAt'],})
    
    articles_df = pd.DataFrame(all_articles)
    articles_df.to_csv(output_file, index=False)
    print(f"Articles saved to {output_file}")

In [79]:
process_athletes_in_chunks(athletes, api_key)

Processing chunk from index 0 to 50...
Fetching articles for Cristiano Ronaldo...
Fetching articles for LeBron James...
Fetching articles for Andrés Iniesta...
Fetching articles for Ronaldinho...
Fetching articles for Karim Benzema...
Fetching articles for Kevin Durant...
Fetching articles for Gerard Piqué...
Fetching articles for Luis Suárez...
Fetching articles for Stephen Curry...
Fetching articles for Sergio Agüero...
Fetching articles for Shaquille O'Neal...
Error fetching articles for Shaquille O'Neal: 400 Client Error: Bad Request for url: https://gnews.io/api/v4/search?q=Shaquille%20O'Neal&from=2020-01-01&to=2021-12-31&lang=en&max=10&apikey=4eff157235e983a5f72db09ead7ecf00
Error fetching articles for Shaquille O'Neal: 400 Client Error: Bad Request for url: https://gnews.io/api/v4/search?q=Shaquille%20O'Neal&from=2020-01-01&to=2021-12-31&lang=en&max=10&apikey=4eff157235e983a5f72db09ead7ecf00
Error fetching articles for Shaquille O'Neal: 400 Client Error: Bad Request for url: htt

articles = pd.read_csv('articles.csv')
articles